In [21]:
import pandas as pd
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

In [22]:
cid = '18ef04bfcc4749dd8e7e23c4e3c114af'
secret = '204361e7fd324bbfb9d32dcbeeb94708'
client_credentials_manager = SpotifyClientCredentials(client_id=cid, client_secret=secret)
sp = spotipy.Spotify(client_credentials_manager = client_credentials_manager)

In [20]:
spotify_data = pd.read_csv('SpotifyFeatures.csv')
spotify_data = spotify_data.drop_duplicates(subset = ['track_id'])
spotify_data.to_csv('SpotifyFeaturesCleaned.csv', index=False)


In [43]:
spotify_data = pd.read_csv('SpotifyFeaturesCleaned.csv')
track_ids = spotify_data['track_id'].to_list();


In [10]:
names = []
samples = []
covers = []
for i in range(0, len(track_ids), 50):
    tracks = sp.tracks(track_ids[i:i+50])['tracks']
    for track in tracks:
        names.append(track['name'])
        samples.append(track['preview_url'])
        covers.append(track['album']['images'][0]['url'] if track['album']['images'] else None)

In [24]:
cover_n_sample_df = pd.DataFrame(list(zip(track_ids, names, samples, covers)), 
               columns =['id', 'name', 'sample', 'cover'])

In [26]:
cover_n_sample_df.to_csv('cover_n_sample.csv')


In [39]:
artist_names = []
artist_ids = []
artist_track_ids = []
for i in range(0, len(track_ids), 50):
    tracks = sp.tracks(track_ids[i:i+50])['tracks']
    for track in tracks:
        artists = track['artists']
        for artist in artists:
            artist_track_ids.append(track['id'])
            artist_names.append(artist['name'])
            artist_ids.append(artist['id'])


In [44]:
song_artist_df = pd.DataFrame(list(zip(artist_names, artist_ids, artist_track_ids)),
               columns =['artist_name', 'artist_id', 'track_id'])

In [41]:
song_artist_df.to_csv('song_n_artist.csv')

In [31]:
artist_ids_unique = list(set(artist_ids))
artists_ids_genres = []
artists_genres = []

for i in range(0, len(artist_ids_unique), 50):
    artists = sp.artists(artist_ids_unique[i:i+50])['artists']
    for artist in artists:
        genres = artist['genres']
        for genre in genres:
            artists_ids_genres.append(artist['id'])
            artists_genres.append(genre)

In [32]:
artists_genres_df = pd.DataFrame(list(zip(artists_ids_genres, artists_genres)),
               columns =['artist_id', 'genre'])

In [33]:
artists_genres_df.to_csv('artists_n_genres.csv')

In [ ]:
from backend.data.database_utils import upload_dataframe_as_table
upload_dataframe_as_table(spotify_data, 'tracks')
upload_dataframe_as_table(song_artist_df, 'song_n_artist')
upload_dataframe_as_table(artists_genres_df, 'artist_n_genres')